In [3]:
from androguard.misc import AnalyzeDex
from androguard.core.api_specific_resources import load_permission_mappings


import timeit
import os
import glob
import json

In [2]:
#Axplorer API-permission mappings
mapping = load_permission_mappings(25) #axplorer mappingleri
axplorer = dict()
for k,v in mapping.items(): 
    api = k.split("(")[0]
    api = api.replace('-','')
    axplorer[api] = v

In [4]:
#Find all dex files
LIB_PATH = os.path.join(os.environ["NLG_ROOT"], "datasets/libraries/repo/")

dex_file_paths = []
for x in os.walk(LIB_PATH):
    for y in glob.glob(os.path.join(x[0], '*.dex')):
        dex_file_paths.append(y)


In [12]:
LIMIT = 20

lib_analysis = {}
for idx, dex_path in enumerate(dex_file_paths):
    if idx > LIMIT:
        break
    path_components = dex_path.split("/")
    lib_name, version = path_components[-3], path_components[-2]
    a, b, dx = AnalyzeDex(dex_path)
    print(dex_path)
    if lib_name not in lib_analysis:
        lib_analysis[lib_name] = {}
    lib_analysis[lib_name][version] = set()
    for dx_class in dx.get_classes():
        for method in dx_class.get_methods():
            if len(method.get_xref_to()):
                for api_call in method.get_xref_to():
                    split_class_name = str(api_call[1])
                    wanted = split_class_name.split("(")[0]
                    wanted = wanted.replace('-', '')
                    wanted = wanted.replace('>','')
                    if wanted in axplorer:
                        for perm in axplorer[wanted]:
                            lib_analysis[lib_name][version].add(perm)



/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/com.android.support::swiperefreshlayout/28.0.0/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.3.1/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.5.3/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.3.0/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.5.2/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.4.0/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG/datasets/libraries/repo/Android/androidx.databinding::databinding-adapters/3.4.1/classes.dex
/Users/huseyinalecakir/Desktop/Security/NLG

In [11]:
lib_analysis

{'com.android.support::swiperefreshlayout': set(),
 'androidx.databinding::databinding-adapters': set(),
 'androidx.percentlayout::percentlayout': set(),
 'androidx.interpolator::interpolator': set(),
 'com.android.support::support-content': set()}